In [3]:
import datetime

In [4]:
def handle_bit_type(bit_value):
    return int.from_bytes(bit_value, byteorder='big')

In [7]:
import pyodbc
import psycopg2
import psycopg2.extras
table_list = ['organisationusertextmessagesetting']

sqls_cnxn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=172.16.1.33\SQL2014DEV;DATABASE=ADB_Prod_Framework_Dev;UID=adb_prod;PWD=adb_prod")

sqls_cnxn.add_output_converter(pyodbc.SQL_BIT, handle_bit_type)
sqls_cursor = sqls_cnxn.cursor()

print('Sql server connection created')

sqlp_cnxn = psycopg2.connect(
        host="adb-dev-pg-cluster.cluster-ch6hownygg05.us-west-2.rds.amazonaws.com",
        database="Complio_Messaging_Dev",
        user="postgres",
        password="Inter$oft2021",
        options="-c search_path=adb_prod_messaging_dev_dbo")

sqlp_cursor = sqlp_cnxn.cursor()

print('Postgres connection created')

#cursor.execute('SELECT * FROM adb_prod_framework_dev.dbo.lkpOrderStatus')

for table_name in table_list:
    column_names = list(map(lambda x: x.column_name.lower(), sqls_cursor.columns(table=table_name)))
  
    #get records from sql server
    sqls_cursor.execute("select * from ADB_Prod_Framework_Dev.dbo."+ table_name)
    
    print('Fetching started for table: ', table_name)
    rows_data = sqls_cursor.fetchall()
    print('Fetching completed for table: ', table_name)
    #print(rows_data)
      
    insert_statment = 'INSERT INTO dbo.{}({}) OVERRIDING SYSTEM VALUE VALUES  %s'.format(table_name.lower(), ', '.join(column_names))
    #print('Insert started for table: ', table_name, datetime.now().strftime("%H:%M:%S"))
    try:
        psycopg2.extras.execute_values(sqlp_cursor, insert_statment, rows_data, page_size = 5000)
        #print('Insert completed for table(Transaction yet to complete): ', table_name)
        sqlp_cursor.connection.commit()
        #print('Transaction commited. Done!!!', datetime.now().strftime("%H:%M:%S"))
    except Exception as e:
        print("Transaction can't be completed, rollback!!", e)
        sqlp_cursor.connection.rollback()
    
sqls_cursor.close()
sqlp_cursor.close()

sqls_cnxn.close()
sqlp_cnxn.close()

Sql server connection created
Postgres connection created
Fetching started for table:  organisationusertextmessagesetting
Fetching completed for table:  organisationusertextmessagesetting
